## Raw code notebook

In [28]:
import pandas as pd
import sqlite3
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.float_format', lambda x: '%.f' % x)

bom_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
ret_info_df = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep = '\t')
ret_reviews_df = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep = '\t', encoding='unicode_escape')
tmdb_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz', index_col=0)
tn_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')
conn = sqlite3.Connection('./zippedData/john/im.db')

## Cleaning the data from 'The Movie Database'

In [29]:
tmdb_df.drop('original_title', axis=1, inplace = True)
tmdb_df['genre_ids'].replace(to_replace= '[]', value = '[7]', inplace = True)
getGenre = {
    '7': 'undecided', '12': 'Adventure', '14': 'Fantasy', '16': 'Animation', '18': 'Drama', '27': 'Horror', '28': 'Action',
    '35': 'Comedy', '36': 'History', '37': 'Western', '53': 'Thriller', '80': 'Crime', '99': 'Documentary', 
    '878': 'Science Fiction','9648': 'Mystery','10402': 'Music','10749':'Romance','10751': 'Family','10752': 'War','10770': 'TV'
}
def convert(glist):
    description = ''
    for num in glist.strip('][').split(', '):
        description += getGenre[num]
        description += ', '
    return description.rstrip(', ')
tmdb_df['genre_ids'] = tmdb_df['genre_ids'].map(lambda x: convert(x))
tmdb_df.drop(['id', 'original_language'], axis=1, inplace=True)
tmdb_df = tmdb_df[['title', 'release_date', 'genre_ids', 'popularity', 'vote_average', 'vote_count']]
tmdb_df.drop_duplicates(inplace=True)

## Cleaning the data from 'The Numbers'

In [4]:
tn_df['release_date'] = pd.to_datetime(tn_df['release_date'])
tn_df['production_budget'] = tn_df['production_budget'].map(lambda x: int(x.replace('$','').replace(',','')))
tn_df['domestic_gross'] = tn_df['domestic_gross'].map(lambda x: int(x.replace('$','').replace(',','')))
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].map(lambda x: int(x.replace('$','').replace(',','')))

## Clean the data from 'Box Office Mojo'

In [5]:
bom_df['foreign_gross'] = bom_df['foreign_gross'].fillna('0')
bom_df['foreign_gross'] = bom_df['foreign_gross'].map(lambda x: x.replace(',', '')).map(lambda x: float(x))
bom_df['domestic_gross'] = bom_df['domestic_gross'].astype(float)

## Merge and clean the differnt tables from 'IMDB'

In [6]:
mvi = pd.read_sql('''
SELECT movie_id, (primary_title) AS title, (start_year) year, (runtime_minutes) AS runtime, genres,
(averagerating) AS avg_rating, numvotes
FROM movie_basics
LEFT JOIN movie_ratings
USING(movie_id)

''', conn)

directors = pd.read_sql('''
SELECT movie_id, (primary_name) AS director
FROM principals
JOIN persons
USING(person_id)
WHERE category = 'director'
''', conn)
directors = directors.drop_duplicates()
directors = directors.groupby('movie_id').agg(lambda x: x.tolist()).reset_index()

actors = pd.read_sql('''
SELECT movie_id, (primary_name) AS actors
FROM principals
JOIN persons
USING(person_id)
WHERE category = 'actor'
ORDER BY movie_id
''', conn)
actors = actors.drop_duplicates()
actors = actors.groupby('movie_id').agg(lambda x: x.tolist()).reset_index()

draft1 = pd.merge(mvi,directors, how = 'left', on = 'movie_id')
sql_df = pd.merge(draft1, actors, how = 'left', on = 'movie_id')
sql_df['runtime'].fillna(sql_df['runtime'].median(), inplace = True)
sql_df.drop('movie_id', axis=1, inplace=True)

## Merge 'Box Office Mojo' with 'The Numbers'

In [ ]:
bomtn = pd.merge(bom_df, tn_df, how='outer', left_on='title', right_on='movie')

bomtn['title'] = bomtn['title'].fillna(bomtn['movie'])
bomtn['domestic_gross_x'] = bomtn['domestic_gross_x'].fillna(bomtn['domestic_gross_y'])
bomtn['foreign_gross'] = bomtn['foreign_gross'].fillna(bomtn['worldwide_gross'] - bomtn['domestic_gross_y'])
bomtn['release_date'] = bomtn['release_date'].fillna(pd.to_datetime(bomtn['year'], format='%Y'))
bomtn.drop(bomtn.columns[[1, 4, 5, 7, 9, 10]], axis=1, inplace=True)
bomtn = bomtn[['title','release_date','production_budget', 'domestic_gross_x', 'foreign_gross']]
bomtn.rename(columns={'domestic_gross_x':'domestic_gross'}, inplace=True)
bomtn['worldwide_gross'] = bomtn['domestic_gross'] + bomtn['foreign_gross']
bomtn.dropna(inplace=True)
bomtn = bomtn[bomtn['worldwide_gross'] != 0.0]

In [31]:
tmdbsql = pd.merge(sql_df, tmdb_df,how ='outer', on='title' )
tmdbsql['avg_rating'] = tmdbsql['avg_rating'].fillna(tmdbsql['vote_average'])
tmdbsql['numvotes'] = tmdbsql['numvotes'].fillna(tmdbsql['vote_count'])
tmdbsql['genres'] = tmdbsql['genres'].fillna(tmdbsql['genre_ids'])

In [33]:
pd.merge(sql_df, tmdb_df,how ='inner', on='title' ).isna().sum()

title              0
year               0
runtime            0
genres           331
avg_rating      3502
numvotes        3502
director        2003
actors          4517
release_date       0
genre_ids          0
popularity         0
vote_average       0
vote_count         0
dtype: int64

In [54]:
run = sql_df[['year', 'runtime']].dropna()
runtime = run.groupby('year').agg('mean').reset_index()
runtime

,year,runtime
0,2010,86
1,2011,87
2,2012,89
3,2013,85
4,2014,85
5,2015,86
6,2016,85
7,2017,86
8,2018,87
9,2019,89
